On commence par la méthode usuelle, mais on rajoute un paramètre sur l'API pour se concentrer sur l'année 2020 : 

In [2]:
import pandas as pd
import geopandas as gpd
import requests
from io import StringIO

In [3]:
def cleaning_json_for_gpd(data):
    for feature in data["features"]:
        properties = feature["properties"]
        for key, value in properties.items():
            if isinstance(value, list):  # Vérifie si c'est une liste
                properties[key] = ", ".join(map(str, value))  # Convertit en chaîne séparée par des virgules
    

In [4]:
BASE_URL_API = "https://apidf-preprod.cerema.fr"
url = BASE_URL_API + f"/dvf_opendata/geomutations/?anneemut=2020&code_insee=59350"
response = requests.get(url)
# Vérification du succès de la requête
if response.status_code == 200:
    data = response.json()  # On récupère les données au format JSON
else:
    print(f"Erreur : {response.status_code}")
mut_gdf = gpd.read_file(StringIO(response.text))
len(mut_gdf)

Skipping field l_codinsee: unsupported OGR type: 5
Skipping field l_idpar: unsupported OGR type: 5
Skipping field l_idparmut: unsupported OGR type: 5
Skipping field l_idlocmut: unsupported OGR type: 5


100

OK, on voit bien qu'on manque qqch. On n'a que 100 observations dans notre dataset pour toute l'année 2020 pour tout Lille. Et c'est exactement le même nombre que pour d'autres grandes villes. Mais regardons du côté de notre variable "data", c'est-à-dire notre requête au format json : 

In [5]:
print(data)

{'type': 'FeatureCollection', 'count': 5674, 'next': 'http://apidf-preprod.cerema.fr/dvf_opendata/geomutations/?anneemut=2020&code_insee=59350&page=2', 'previous': None, 'features': [{'id': 8341428, 'type': 'Feature', 'geometry': {'type': 'MultiPolygon', 'coordinates': [[[[3.05323, 50.637196], [3.053166, 50.637149], [3.053166, 50.637149], [3.05299, 50.637022], [3.052832, 50.636907], [3.052832, 50.636907], [3.052832, 50.636907], [3.052832, 50.636907], [3.052832, 50.636907], [3.052832, 50.636907], [3.052832, 50.636907], [3.052832, 50.636907], [3.052832, 50.636907], [3.052799, 50.636926], [3.052799, 50.636926], [3.052581, 50.637047], [3.052581, 50.637047], [3.052581, 50.637047], [3.052581, 50.637047], [3.052581, 50.637047], [3.052581, 50.637047], [3.052581, 50.637047], [3.052581, 50.637047], [3.052581, 50.637047], [3.052714, 50.637142], [3.052789, 50.637196], [3.052843, 50.637236], [3.052916, 50.637289], [3.052916, 50.637289], [3.052916, 50.637289], [3.052916, 50.637289], [3.052916, 50.63

Quand on regarde directement data, on a une variable 'count' qui semble donner le nombre d'observations. Par exemple, pour Lille on en a 5674, et pour la commune d'à côté (code 59351) on en a seulement 7. 

En fait, ce n'est pas pandas qui nous supprime des observations, mais l'API qui nous renvoie les observations par page de 100. Ainsi, à chaque fois qu'on requête l'API et qu'on obtient pas l'intégralité des observations, la réponse ("data") contient une clé 'next' qui renvoie une autre url qu'on peut utiliser pour obtenir la page suivante (s'il n'y a pas de page suivante, la valeur est None). En conséquence, on peut regarder le nombre d'observations à Lille année par année avec le programme suivant : 

In [6]:
BASE_URL_API = "https://apidf-preprod.cerema.fr"
code_insee = "59350"
for i in range(0,15):
    url = BASE_URL_API + f"/dvf_opendata/geomutations/?anneemut={2010 + i}&code_insee=59350"
    response = requests.get(url)
    data = response.json()
    print(2010+i, data["count"])


2010 0
2011 0
2012 0
2013 0
2014 4538
2015 0
2016 5376
2017 6166
2018 6287
2019 6587
2020 5674
2021 6242
2022 5814
2023 4514
2024 1643


Essayons maintenant de récupérer un dataframe contenant toutes les observations d'une année (ici, 2020). Mais pour vérifier qu'on ne concatène pas le même dataframe en boucle, et parce que la médiane du prix d'achat est une valeur intéressante, on va regarder la médiane de notre df de 100 observations. 

In [7]:
mut_gdf['valeurfonc']  = mut_gdf["valeurfonc"].astype(float)
mut_gdf['valeurfonc'].median()

np.float64(189600.0)

On peut faire tourner notre programme :

In [13]:
url = "https://apidf-preprod.cerema.fr/dvf_opendata/geomutations/?anneemut=2020&code_insee=59350&page_size=300"
response = requests.get(url)
data = response.json()
cleaning_json_for_gpd(data)
mut_gdf_tempo = gpd.GeoDataFrame.from_features(data["features"])
col_mut = mut_gdf_tempo.columns
mut_gdf = gpd.GeoDataFrame(columns=col_mut)

while url != None : 
    response = requests.get(url)
    if response.status_code == 200:
            data = response.json()
            url = data['next']
            cleaning_json_for_gpd(data)
            mut_gdf_tempo = gpd.GeoDataFrame.from_features(data["features"])
            mut_gdf = pd.concat([mut_gdf, mut_gdf_tempo], axis=0)
            print(data['next'])
            
    else:
        print(f"Erreur : {response.status_code}")
        url = None
    

http://apidf-preprod.cerema.fr/dvf_opendata/geomutations/?anneemut=2020&code_insee=59350&page=2&page_size=300
http://apidf-preprod.cerema.fr/dvf_opendata/geomutations/?anneemut=2020&code_insee=59350&page=3&page_size=300
http://apidf-preprod.cerema.fr/dvf_opendata/geomutations/?anneemut=2020&code_insee=59350&page=4&page_size=300
http://apidf-preprod.cerema.fr/dvf_opendata/geomutations/?anneemut=2020&code_insee=59350&page=5&page_size=300
http://apidf-preprod.cerema.fr/dvf_opendata/geomutations/?anneemut=2020&code_insee=59350&page=6&page_size=300
http://apidf-preprod.cerema.fr/dvf_opendata/geomutations/?anneemut=2020&code_insee=59350&page=7&page_size=300
http://apidf-preprod.cerema.fr/dvf_opendata/geomutations/?anneemut=2020&code_insee=59350&page=8&page_size=300
http://apidf-preprod.cerema.fr/dvf_opendata/geomutations/?anneemut=2020&code_insee=59350&page=9&page_size=300
http://apidf-preprod.cerema.fr/dvf_opendata/geomutations/?anneemut=2020&code_insee=59350&page=10&page_size=300
http://ap

On vérifie la taille du dataframe ainsi obtenu

In [15]:
mut_gdf.shape

(5674, 22)

Et on va regarder la médiane de ce nouveau dataframe : 

In [14]:
mut_gdf['valeurfonc']  = mut_gdf["valeurfonc"].astype(float)
mut_gdf["valeurfonc"].median()

np.float64(180600.0)

L'API permet désormais de boucler pour récupérer toutes les données, mais il semble que ça beugue parfois (sans que je comprenne vraiment pourquoi...) ; à résoudre avec un chargé de TD